In [1]:
import pandas as pd
import argparse
import pdb
import os
import numpy as np
import glob
import tqdm
from PIL import Image
import numpy as np
import cv2
import torch
from torchvision import transforms
from transformers import ViTImageProcessor, ViTForImageClassification
from datasets import load_metric, Dataset, DatasetDict, load_dataset
import datasets
from transformers import TrainingArguments
from transformers import Trainer

In [2]:
device = "cpu"

model_name_or_path = './vit-base-beans'
feature_extractor = ViTImageProcessor.from_pretrained(model_name_or_path, device = device)


In [3]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x.convert('RGB') for x in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

/var/folders/xw/b9pjrnqs1qb383_fxg5k6lpm0000gn/T/ipykernel_30480/2379461522.py:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [5]:
ds = load_dataset("imagefolder", data_dir='statista-images')
prepared_ds = ds.with_transform(transform)
labels = prepared_ds['train'].features['label'].names

Resolving data files:   0%|          | 0/1518 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1206 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/255 [00:00<?, ?it/s]

Found cached dataset imagefolder (/Users/akankshitadash/.cache/huggingface/datasets/imagefolder/default-a0262678a4de68aa/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
#Train model
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

training_args = TrainingArguments(
    output_dir="./vit-base-beans",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=4,
    fp16=False,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

RuntimeError: TensorBoardCallback requires tensorboard to be installed. Either update your PyTorch version or install tensorboardX.

In [28]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

test_results = trainer.predict(prepared_ds["test"])
predicted_class = np.argmax(test_results.predictions, axis=1).astype(int)

#Save prediction results
filename = [ds['test'][x]['image'].filename for x in range(len(ds['test']))]
mapping = {i: labels[i] for i in range(len(labels))}

/opt/homebrew/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


UnidentifiedImageError: cannot identify image file '/Users/akankshitadash/Desktop/Stanford/Quarter3/CS399/WiT/statista_sample_cat/test/nolabel/train-0-304.jpg'

In [25]:
results_df = pd.DataFrame({"filename": filename, "pred": [mapping[int(x)] for x in list(predicted_class)]}).sort_values('filename')

In [29]:
results_df

,filename,pred
0,/content/drive/MyDrive/CS399/statista_sample_c...,pie
1,/content/drive/MyDrive/CS399/statista_sample_c...,horizontal
2,/content/drive/MyDrive/CS399/statista_sample_c...,horizontal
3,/content/drive/MyDrive/CS399/statista_sample_c...,vertical
4,/content/drive/MyDrive/CS399/statista_sample_c...,horizontal
...,...,...
6288,/content/drive/MyDrive/CS399/statista_sample_c...,other
6289,/content/drive/MyDrive/CS399/statista_sample_c...,other
6290,/content/drive/MyDrive/CS399/statista_sample_c...,other
6291,/content/drive/MyDrive/CS399/statista_sample_c...,other


In [31]:
results_df['filename_short'] = results_df['filename'].apply(lambda x: x.split('/')[-1])


In [34]:
new_df = results_df[results_df['pred']=='other']

In [37]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 6273 to 6292
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   filename        20 non-null     object
 1   pred            20 non-null     object
 2   filename_short  20 non-null     object
dtypes: object(3)
memory usage: 640.0+ bytes


In [ ]:
results_df.to_csv(os.path.join('output/chart_preds', "classification_results.csv"))


In [6]:
import os

In [8]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/MyDrive/CS399')

Mounted at /content/drive


In [5]:
! zip -r 1.zip statista_sample_cat/

	zip warning: name not matched: statista_sample_cat/

zip error: Nothing to do! (try: zip -r 1.zip . -i statista_sample_cat/)
